In [107]:
import requests
import re
import numpy as np
import time
from bs4 import BeautifulSoup
import pandas as pd
import json 
import requests
import datetime
from datetime import date
pd.set_option('max_colwidth', 800)

In [2]:
from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key='a539d7df2c7b43e1ac4d12f386d901e8')

In [3]:
x = newsapi.get_top_headlines(country='us', page_size=50, category = 'health')

In [4]:
nyt_api_key = '13bd501bc77542a58e2e6678619b0d60'

In [9]:
today = int(str(date.today()).replace('-',''))
last_week = int(str(date.today() - datetime.timedelta(days = 14)).replace('-',''))

In [5]:
def NYT_title_clean(df):
    titles = []
    for index, row in df.iterrows():
        title = row.headline['main']
        titles.append(title)
    df['title'] = titles
    return df

In [6]:
def NYT_dropped_rows(df):
    df = df.drop(['blog','byline','headline','keywords','multimedia','news_desk','print_page','score','type_of_material','uri'],axis =1)
    df.pub_date = pd.to_datetime(df.pub_date).dt.date
    df.word_count = round(df.word_count / 150)
    df.document_type = 'text'
    df['expertise'] = 'Intermediate'
    return df

In [7]:
def NYT_dataframe_clean(df):
    dataframe = NYT_title_clean(df)
    dataframe = NYT_dropped_rows(dataframe)
    return dataframe

In [111]:
def NYT_api_call_section_based(section, source, page, start, end, key):
    url = 'http://api.nytimes.com/svc/search/v2/articlesearch.json?fq=section_name:({section_name})&page={page}&source:({source})&begin_date={start}&end_date={end}&api-key={api}'.format(section_name = section, page = page, source = source, start = start, end = end, api = key)
    resp = requests.get(url=url)
    data = json.loads(resp.text)
    df = pd.DataFrame(data['response']['docs'])
    df = NYT_dataframe_clean(df)
    df['param'] = section
    df['image_url'] = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSHEtiVXw8Wi1tp56Nzd5rH_EoOAJA2RInEWvf5h5CQ-6O_YZp7dw'
    return df
    
    

In [161]:
def NYT_api_call_parameter_based(param, start, end, key):
    url = 'http://api.nytimes.com/svc/search/v2/articlesearch.json?fq={param}&facet_field=day_of_week&begin_date={start}&end_date={end}&api-key={api}'.format(param = param, start = start, end = end, api = key)
    resp = requests.get(url=url)
    data = json.loads(resp.text)
    df = pd.DataFrame(data['response']['docs'])
    df = NYT_dataframe_clean(df)
    df['param'] = param
    df['image_url'] = 'https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg'
    return df
    

In [162]:
def NYT_api_call_parameter_ALLTIME(param, page, key):
    url = 'http://api.nytimes.com/svc/search/v2/articlesearch.json?q={param}&page={page}&sort=newest&&api-key={api}'.format(param = param, page = page, api = key)
    resp = requests.get(url=url)
    data = json.loads(resp.text)
    df = pd.DataFrame(data['response']['docs'])
    df = NYT_dataframe_clean(df)
    df['param'] = param
    df['image_url'] = 'https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg'
    return df

In [118]:
NYT_api_call_parameter_ALLTIME('ketogenic',0,nyt_api_key)

,_id,abstract,document_type,pub_date,section_name,snippet,source,web_url,word_count,title,expertise,param,image_url
0,5be5a7fa00a1bc2872e930db,NaN,text,2018-11-09,NaN,"Make Dorie Greenspan’s latest recipe, an autumnal riff on an Eton mess with cranberry jam, lemon curd and raspberries.",The New York Times,https://www.nytimes.com/2018/11/09/dining/what-to-cook-this-weekend.html,5.0,What to Cook This Weekend,Intermediate,ketogenic,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg
1,5ad449f947de81a901223348,NaN,text,2018-04-16,Baseball,"The Mets starter Jacob deGrom strikes a contrast in physique and habits with his fellow starter, the muscular and health-conscious Noah Syndergaard, known as Thor.",The New York Times,https://www.nytimes.com/2018/04/16/sports/baseball/mets-degrom.html,8.0,Not Necessarily Thor: One Mets Ace Is Not Like the Other,Intermediate,ketogenic,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg
2,5a830b9cd4211f00015c0aca,NaN,text,2018-02-13,NaN,A delicious new crop of cocoa-based products boast surprising wellness benefits.,The New York Times,https://www.nytimes.com/2018/02/13/t-magazine/healthier-chocolate.html,4.0,"The New, Healthier Chocolate",Intermediate,ketogenic,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg
3,5a42a15795d0e0246f21e2d0,NaN,text,2017-12-26,Pro Basketball,"He is the beating heart of British basketball and his journey has taken him from the Bronx to Buckingham Palace. And, at 44, he is still going. “In his mind, age doesn’t apply.”",The New York Times,https://www.nytimes.com/2017/12/26/sports/basketball/fabulous-flournoy.html,19.0,The Fantastical Adventures of Fabulous Flournoy,Intermediate,ketogenic,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg
4,59ca172195d0e0246f20c5f2,NaN,text,2017-09-26,NaN,"At the height of the 2016 election, exaggerated reports of a juvenile sex crime brought a media maelstrom to Twin Falls — one the Idaho city still hasn’t recovered from.",The New York Times,https://www.nytimes.com/2017/09/26/magazine/how-fake-news-turned-a-small-town-upside-down.html,37.0,How Fake News Turned a Small Town Upside Down,Intermediate,ketogenic,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg
5,578cd2bd38f0d83596d5497f,"Using yoga-posing pears and bike-riding melons, a new app deconstructs your food, giving you a trove of information not found on a typical food label.",text,2016-07-18,NaN,"Using yoga-posing pears and bike-riding melons, a new app deconstructs your food, giving you a trove of information not found on a typical food label.",The New York Times,https://well.blogs.nytimes.com/2016/07/18/an-app-to-deconstruct-your-food/,7.0,An App to Deconstruct Your Food,Intermediate,ketogenic,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg
6,56ddd6b238f0d810772443ea,"The health benefits of skipping meals are growing more apparent, though critics argue that the idea is still impractical for most people.",text,2016-03-07,NaN,"The health benefits of skipping meals are growing more apparent, though critics argue that the idea is still impractical for most people.",The New York Times,https://well.blogs.nytimes.com/2016/03/07/intermittent-fasting-diets-are-gaining-acceptance/,11.0,Fasting Diets Are Gaining Acceptance,Intermediate,ketogenic,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg
7,5641dc5838f0d80e74ff9934,NaN,text,2015-11-10,NaN,"Is “biohacking” just a fad? Or can data-driven diets help us become an improved, happier species?",The New York Times,https://www.nytimes.com/2015/11/15/magazine/you-only-better.html,18.0,"You, Only Better",Intermediate,ketogenic,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg
8,54eda80138f0d828b27a128f,Most athletes believe high-carb diets are the b

In [130]:
df = NYT_api_call_parameter_ALLTIME('keto',0,nyt_api_key)
time.sleep(5)
df2 = NYT_api_call_parameter_ALLTIME('ketogenic',0,nyt_api_key)
df = df.append(df2)

In [171]:
def diet_NYT():
    empty = pd.DataFrame()
    for word in ['keto','ketogenic','paleo','vegan','vegetarian']:
        df = NYT_api_call_parameter_ALLTIME(word,0,nyt_api_key)
        empty = empty.append(df, sort=True)
        time.sleep(2)
    empty = empty.drop(['abstract','section_name'],axis = 1)
    empty = empty.rename(index=str, columns={"_id": "source_id", "document_type": "medium",'pub_date':'date','snippet':'description','word_count':'length'})
    return empty

In [150]:
df = NYT_api_call_parameter_ALLTIME('keto',0,nyt_api_key)

In [172]:
diet_NYT()

,source_id,medium,expertise,image_url,param,date,description,source,title,web_url,length
0,5bb7d84100a1bc2872e8c295,text,Intermediate,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg,keto,2018-10-05,"The electric pressure cooker is well suited for many Indian dishes, and a slew of new cookbooks are seizing on its popularity.",The New York Times,Indian Cooks Embrace the Instant Pot,https://www.nytimes.com/2018/10/05/dining/instant-pot-indian.html,8.0
1,5b583c1600a1bc2872e7e210,text,Intermediate,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg,keto,2018-07-25,Inside the growth of the most controversial brand in the wellness industry.,The New York Times,How Goop’s Haters Made Gwyneth Paltrow’s Company Worth $250 Million,https://www.nytimes.com/2018/07/25/magazine/big-business-gwyneth-paltrow-wellness.html,54.0
2,5b54f05400a1bc2872e7db42,text,Intermediate,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg,keto,2018-07-22,A small storefront in Manhattan houses a very deep knowledge of a very narrow subject: books about food.,The New York Times,This Bookstore Is Stuffed With Just One Thing: Cookbooks,https://www.nytimes.com/2018/07/22/books/bookstore-cookbooks-kitchen-arts-and-letters.html,5.0
3,5aa2e7cb47de81a90120c7c9,text,Intermediate,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg,keto,2018-03-09,"Some of the biggest names in dieting, organic agriculture and preventive medicine died at surprisingly young ages.",The New York Times,The Secret to a Longer Life? Don’t Ask These Dead Longevity Researchers,https://www.nytimes.com/2018/03/09/opinion/sunday/longevity-pritikin-atkins.html,10.0
4,5a7c1fc8d4211f00015bff91,text,Intermediate,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg,keto,2018-02-08,A global network of live-work spaces is springing up to serve this new breed of millennial wanderer.,The New York Times,"When You’re a ‘Digital Nomad,’ the World Is Your Office",https://www.nytimes.com/2018/02/08/magazine/when-youre-a-digital-nomad-the-world-is-your-office.html,25.0
5,5a1d33d395d0e0246f218e5f,text,Intermediate,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg,keto,2017-11-28,"New cookbooks to make you feel good, along with books of cakes and cookies to make you feel happy. And, for the brave, recipes for not-so-awful offal.",The New York Times,"Lots of Healthy Options, but Don’t Forget Dessert!",https://www.nytimes.com/2017/11/28/books/review/cookbooks-baking-offal-healthy-eating.html,17.0
6,51d3e8cd7e0d9c0839d2db71,text,Intermediate,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg,keto,2013-07-03,"Jana Romanova traveled to Georgia to reconnect with long-lost relatives, and along the way, she turned the very concept of “relative” on its head.",The New York Times,"From Russia to Georgia, a Lesson in Relativity",https://lens.blogs.nytimes.com/2013/07/03/its-all-relative-forever/,6.0
7,54219cc738f0d83c6e5b6fdb,text,Intermediate,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg,keto,2010-12-24,"A lively White Columns annual review, gallery shows featuring Keltie Ferris and Marie-Louise Motesiczky, and a very old mosaic floor at the Met.",The New York Times,"&#8216;THE ROMAN MOSAIC From Lod, Israel&#8217;",https://www.nytimes.com/2010/12/24/arts/design/24galleries-THEROMANMOSA_RVW.html,2.0
8,5420a83338f0d81f7a60063f,text,Intermediate,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg,keto,2010-12-12,"“Unfortunately, the proliferation of quack dietary treatments has made gaining acceptance of nutritional therapies that actually have solid science behind them an uphill battle.”",The New York Times,"Letter: A Big, Fat Miracle",https://www.nytimes.com/2010/12/12/magazine/12Letters-t-ABIGFATMIR

In [127]:
df2 = NYT_api_call_parameter_ALLTIME('ketogenic',0,nyt_api_key)

In [128]:
df = df.append(df2)

In [129]:
df

,_id,abstract,document_type,pub_date,section_name,snippet,source,web_url,word_count,title,expertise,param,image_url
0,5bb7d84100a1bc2872e8c295,NaN,text,2018-10-05,NaN,"The electric pressure cooker is well suited for many Indian dishes, and a slew of new cookbooks are seizing on its popularity.",The New York Times,https://www.nytimes.com/2018/10/05/dining/instant-pot-indian.html,8.0,Indian Cooks Embrace the Instant Pot,Intermediate,keto,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg
1,5b583c1600a1bc2872e7e210,NaN,text,2018-07-25,NaN,Inside the growth of the most controversial brand in the wellness industry.,The New York Times,https://www.nytimes.com/2018/07/25/magazine/big-business-gwyneth-paltrow-wellness.html,54.0,How Goop’s Haters Made Gwyneth Paltrow’s Company Worth $250 Million,Intermediate,keto,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg
2,5b54f05400a1bc2872e7db42,NaN,text,2018-07-22,NaN,A small storefront in Manhattan houses a very deep knowledge of a very narrow subject: books about food.,The New York Times,https://www.nytimes.com/2018/07/22/books/bookstore-cookbooks-kitchen-arts-and-letters.html,5.0,This Bookstore Is Stuffed With Just One Thing: Cookbooks,Intermediate,keto,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg
3,5aa2e7cb47de81a90120c7c9,NaN,text,2018-03-09,Sunday Review,"Some of the biggest names in dieting, organic agriculture and preventive medicine died at surprisingly young ages.",The New York Times,https://www.nytimes.com/2018/03/09/opinion/sunday/longevity-pritikin-atkins.html,10.0,The Secret to a Longer Life? Don’t Ask These Dead Longevity Researchers,Intermediate,keto,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg
4,5a7c1fc8d4211f00015bff91,NaN,text,2018-02-08,NaN,A global network of live-work spaces is springing up to serve this new breed of millennial wanderer.,The New York Times,https://www.nytimes.com/2018/02/08/magazine/when-youre-a-digital-nomad-the-world-is-your-office.html,25.0,"When You’re a ‘Digital Nomad,’ the World Is Your Office",Intermediate,keto,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg
5,5a1d33d395d0e0246f218e5f,NaN,text,2017-11-28,Book Review,"New cookbooks to make you feel good, along with books of cakes and cookies to make you feel happy. And, for the brave, recipes for not-so-awful offal.",The New York Times,https://www.nytimes.com/2017/11/28/books/review/cookbooks-baking-offal-healthy-eating.html,17.0,"Lots of Healthy Options, but Don’t Forget Dessert!",Intermediate,keto,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg
6,51d3e8cd7e0d9c0839d2db71,"Jana Romanova traveled to Georgia to reconnect with long-lost relatives, and along the way, she turned the very concept of “relative” on its head.",text,2013-07-03,NaN,"Jana Romanova traveled to Georgia to reconnect with long-lost relatives, and along the way, she turned the very concept of “relative” on its head.",The New York Times,https://lens.blogs.nytimes.com/2013/07/03/its-all-relative-forever/,6.0,"From Russia to Georgia, a Lesson in Relativity",Intermediate,keto,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg
7,54219cc738f0d83c6e5b6fdb,NaN,text,2010-12-24,Art & Design,"A lively White Columns annual review, gallery shows featuring Keltie Ferris and Marie-Louise Motesiczky, and a very old mosaic floor at the Met.",The New York Times,https://www.nytimes.com/2010/12/24/arts/design/24galleries-THEROMANMOSA_RVW.html,2.0,"&#8216;THE ROMAN MOSAIC From Lod, Israel&#8217;",Intermediate,keto,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg
8,5420a83338f0d81f7a60063f,NaN,text,2010-12-12,NaN,"“Unfortunately, the proliferation of quack dietary treatments has made gaining acceptance of nutritional therapies that actually 

In [ ]:
# df2 = NYT_api_call('Health', 'The New York Times',0, last_week, today, nyt_api_key)
# df =  NYT_api_call('Health', 'The New York Times',1, last_week, today, nyt_api_key)
# df = df.append(df2)

In [140]:
def general_health():
    empty = pd.DataFrame()
    for page in [0,1]:
        df = NYT_api_call_section_based('Health', 'The New York Times',page, last_week, today, nyt_api_key)
        empty = empty.append(df)
        time.sleep(2)
    return empty 

In [153]:
def intermediate_search():
    df1 = general_health()
    df2 = diet_NYT()
    frames = [df1,df2]
    result = pd.concat(frames)
    return result

In [185]:
image_dict = {'keto': 'https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg',
             'ketogenic': 'https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg',
             'paleo': 'https://images.agoramedia.com/everydayhealth/gcms/Can-the-Paleo-Diet-Help-Diabetes-1440x810.jpg',
            'paleolithic': 'https://images.agoramedia.com/everydayhealth/gcms/Can-the-Paleo-Diet-Help-Diabetes-1440x810.jpg',
             'vegan': 'https://familydoctor.org/wp-content/uploads/2011/09/shutterstock_413417941-705x468.jpg',
             'vegetarian': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQo-azo8tn-BcljdjG3wp5Oub2vmC3OvDJ2-EXIhxB5dG3EJAq-'}

In [186]:
def NYT_api_call_parameter_ALLTIME(param, page, key):
    url = 'http://api.nytimes.com/svc/search/v2/articlesearch.json?q={param}&page={page}&sort=newest&&api-key={api}'.format(param = param, page = page, api = key)
    resp = requests.get(url=url)
    data = json.loads(resp.text)
    df = pd.DataFrame(data['response']['docs'])
    df = NYT_dataframe_clean(df)
    df['param'] = param
    df['image_url'] = image_dict[param]
    return df

def diet_NYT():
    empty = pd.DataFrame()
    for word in ['keto','ketogenic','paleo','paleolithic','vegan','vegetarian']:
        df = NYT_api_call_parameter_ALLTIME(word,0,nyt_api_key)
        empty = empty.append(df, sort=True)
        time.sleep(2)
    empty = empty.drop(['abstract','section_name'],axis = 1)
    empty = empty.rename(index=str, columns={"_id": "source_id", "document_type": "medium",'pub_date':'date','snippet':'description','word_count':'length'})
    return empty

# def general_health():
#     empty = pd.DataFrame()
#     for page in [0,1]:
#         df = NYT_api_call_section_based('Health', 'The New York Times',page, last_week, today, nyt_api_key)
#         empty = empty.append(df)
#         time.sleep(2)
#     return empty

def intermediate_search():
    # df1 = general_health()
    df2 = diet_NYT()
    # frames = [df1,df2]
    # result = pd.concat(frames)
    return df2 #result

In [187]:
intermediate_search()

,source_id,medium,expertise,image_url,param,date,description,source,title,web_url,length
0,5bb7d84100a1bc2872e8c295,text,Intermediate,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg,keto,2018-10-05,"The electric pressure cooker is well suited for many Indian dishes, and a slew of new cookbooks are seizing on its popularity.",The New York Times,Indian Cooks Embrace the Instant Pot,https://www.nytimes.com/2018/10/05/dining/instant-pot-indian.html,8.0
1,5b583c1600a1bc2872e7e210,text,Intermediate,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg,keto,2018-07-25,Inside the growth of the most controversial brand in the wellness industry.,The New York Times,How Goop’s Haters Made Gwyneth Paltrow’s Company Worth $250 Million,https://www.nytimes.com/2018/07/25/magazine/big-business-gwyneth-paltrow-wellness.html,54.0
2,5b54f05400a1bc2872e7db42,text,Intermediate,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg,keto,2018-07-22,A small storefront in Manhattan houses a very deep knowledge of a very narrow subject: books about food.,The New York Times,This Bookstore Is Stuffed With Just One Thing: Cookbooks,https://www.nytimes.com/2018/07/22/books/bookstore-cookbooks-kitchen-arts-and-letters.html,5.0
3,5aa2e7cb47de81a90120c7c9,text,Intermediate,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg,keto,2018-03-09,"Some of the biggest names in dieting, organic agriculture and preventive medicine died at surprisingly young ages.",The New York Times,The Secret to a Longer Life? Don’t Ask These Dead Longevity Researchers,https://www.nytimes.com/2018/03/09/opinion/sunday/longevity-pritikin-atkins.html,10.0
4,5a7c1fc8d4211f00015bff91,text,Intermediate,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg,keto,2018-02-08,A global network of live-work spaces is springing up to serve this new breed of millennial wanderer.,The New York Times,"When You’re a ‘Digital Nomad,’ the World Is Your Office",https://www.nytimes.com/2018/02/08/magazine/when-youre-a-digital-nomad-the-world-is-your-office.html,25.0
5,5a1d33d395d0e0246f218e5f,text,Intermediate,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg,keto,2017-11-28,"New cookbooks to make you feel good, along with books of cakes and cookies to make you feel happy. And, for the brave, recipes for not-so-awful offal.",The New York Times,"Lots of Healthy Options, but Don’t Forget Dessert!",https://www.nytimes.com/2017/11/28/books/review/cookbooks-baking-offal-healthy-eating.html,17.0
6,51d3e8cd7e0d9c0839d2db71,text,Intermediate,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg,keto,2013-07-03,"Jana Romanova traveled to Georgia to reconnect with long-lost relatives, and along the way, she turned the very concept of “relative” on its head.",The New York Times,"From Russia to Georgia, a Lesson in Relativity",https://lens.blogs.nytimes.com/2013/07/03/its-all-relative-forever/,6.0
7,54219cc738f0d83c6e5b6fdb,text,Intermediate,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg,keto,2010-12-24,"A lively White Columns annual review, gallery shows featuring Keltie Ferris and Marie-Louise Motesiczky, and a very old mosaic floor at the Met.",The New York Times,"&#8216;THE ROMAN MOSAIC From Lod, Israel&#8217;",https://www.nytimes.com/2010/12/24/arts/design/24galleries-THEROMANMOSA_RVW.html,2.0
8,5420a83338f0d81f7a60063f,text,Intermediate,https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg,keto,2010-12-12,"“Unfortunately, the proliferation of quack dietary treatments has made gaining acceptance of nutritional therapies that actually have solid science behind them an uphill battle.”",The New York Times,"Letter: A Big, Fat Miracle",https://www.nytimes.com/2010/12/12/magazine/12Letters-t-ABIGFATMIR